# UKIRT Hemisphere Survey (UHS)

This notebook takes in the downloaded survey and filters it per field creating new fits files for each field.

In [16]:
import subprocess

from astropy.table import Table
from astropy import units as u

import numpy as np

from pymoc import MOC

from herschelhelp_internal.utils import inMoc

In [24]:
#TODO: Maybe check for edge effects and add add extra leeway
fields= [
# Field                RAtyp    DECtyp   RAmin   RAmax   DECmin  DECmax
['AKARI-NEP',          274.65,  65.796,  263.6,  266.5,  68.48,  69.51 ],
['AKARI-SEP',          72.231, -54.380,   66.1,  75.5,  -55.96, -51.61 ],
['Bootes',             216.43,  32.401,  215.7,  220.7,  32.12,  36.18 ],
['CDFS-SWIRE',         51.022, -29.818,   50.7,  55.5,  -30.50, -25.92 ],
['COSMOS',             149.29,  1.0821,  148.6,  151.6,  0.72,   3.71  ],
['EGS',                217.27,  53.644,  212.3,  217.6,  51.08,  54.31 ],
['ELAIS-N1',           247.09,  55.175,  237.9,  247.9,  52.37,  57.59 ],
['ELAIS-N2',           248.42,  39.127,  246.1,  252.3,  39.02,  43.02 ],
['ELAIS-S1',           7.1062, -43.863,    6.3,  11.3,  -45.60, -41.53 ],
['GAMA-09',            129.07, -2.2317,  127.1,  142.2, -2.53,   3.55  ],
['GAMA-12',            172.84,  -0.482,  172.2,  187.4, -3.54,   2.59  ],
['GAMA-15',            211.75, -2.2857,  209.9,  225.3, -2.57,   3.5   ],
['HDF-N',              190.25,  62.205,  188.0,  190.4,  61.69,  62.78 ],
['Herschel-Stripe-82', 353.75, -7.1089,  348.3,  19.1,  -9.5,    9.25  ],
['Lockman-SWIRE',      161.94,  59.056,  154.7,  167.8,  54.95,  60.89 ],
['NGP',                192.89,  22.099,  189.8,  209.3,  21.62,  36.16 ],
['SA13',               197.89,  42.440,  197.5,  198.6,  42.34,  43.09 ],
['SGP',                334.29, -34.503,  333.5,  28.2,  -37.1,  -26.0  ],
['SPIRE-NEP',          266.33,  68.790,  264.6,  275.4,  64.41,  68.63 ],
['SSDF',               341.57, -59.186,  340.5,  2.8,   -61.2,  -48.6  ],
['xFLS',               261.38,  58.018,  255.6,  262.5,  57.85,  60.84 ],
['XMM-13hr',           203.31,  37.474,  202.8,  204.4,  37.31,  38.52 ],
['XMM-LSS',            32.941, -6.0229,   32.1,  38.2,  -7.53,  -1.52  ]]
   
    
#Has a field been removed?  ( RA BETWEEN 13.4 AND 36.3 AND DEC BETWEEN -2.32 AND 2.49 )


In [30]:
uhs = Table.read("data/UHS-DR1_HELP.fits")

In [38]:
uhs["RA"].unit = u.rad
uhs["DEC"].unit = u.rad
uhs["RA"].convert_unit_to(u.deg)
uhs["DEC"].convert_unit_to(u.deg)

In [39]:
uhs[:10].show_in_notebook()

idx,SOURCEID,CUEVENTID,FRAMESETID,RA,DEC,CX,CY,CZ,HTMID,L,B,LAMBDA,ETA,PRIORSEC,MERGEDCLASSSTAT,MERGEDCLASS,PSTAR,PGALAXY,PNOISE,PSATURATED,EBV,AJ,JHALLMAG,JHALLMAGERR,JPETROMAG,JPETROMAGERR,JPSFMAG,JPSFMAGERR,JSERMAG2D,JSERMAG2DERR,JAPERMAG3,JAPERMAG3ERR,JAPERMAG4,JAPERMAG4ERR,JAPERMAG6,JAPERMAG6ERR,JGAUSIG,JELL,JPA,JERRBITS,JDEBLEND,JCLASS,JCLASSSTAT,JPPERRBITS,JSEQNUM,JXI,JETA
,,,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,459598275336,46557,459561520602,249.091395712,41.8277092465,-0.265929128626,-0.696085867558,0.666892917592,15320905220532,66.0875770096,42.2764334789,42.0871194535,31.4778183978,459561576148,-2.10839,-1,0.993865,0.00306749,0.00306749,0.0,0.00753558,0.00669913,17.8318,0.046405,17.7934,0.0700357,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,17.7371,0.0377347,17.7535,0.0474782,17.8746,0.0965107,1.22327,0.185815,105.997,0,--,-1,-1.49085,4194304,1,0.0,0.0
1,459598275337,46557,459561520602,249.090484529,41.7588548997,-0.266226030817,-0.696829786242,0.665996959093,15320721515998,65.994515431,42.2761564233,42.142361292,31.4224044753,459561576148,-11.0131,0,0.00306749,0.00306749,0.993865,0.0,0.00664529,0.00590767,17.602,0.029693,17.5066,0.0626612,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,17.4364,0.0293875,17.4445,0.0363109,17.4465,0.065462,1.30137,0.169551,62.1415,0,--,0,-7.78744,4194304,2,0.0,0.0
2,459598275338,46557,459561520602,249.089374961,41.6795933398,-0.266568101206,-0.697684607174,0.664964387265,15320734749549,65.8873879373,42.2757914461,42.2058919606,31.3584436703,459561576148,3.93553,1,0.00306749,0.993865,0.00306749,0.0,0.00586879,0.00521736,18.6699,0.132642,18.8291,0.241219,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,18.7908,0.0960274,18.7726,0.12005,18.8296,0.235783,1.06449,0.089888,59.3155,0,--,1,2.78284,4194304,3,0.0,0.0
3,459598275339,46557,459561520602,249.089224553,41.6707234215,-0.266606671154,-0.697780061623,0.664848756108,15320734616225,65.8753992885,42.2757641249,42.2129876315,31.3512565069,459561576148,0.675435,-1,0.993865,0.00306749,0.00306749,0.0,0.00583052,0.00518334,15.8568,0.0102708,15.82,0.014632,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,15.8035,0.00881926,15.7949,0.00981466,15.7996,0.0156522,1.59317,0.0841488,170.574,0,--,-1,0.477605,4194304,4,0.0,0.0
4,459598275340,46557,459561520602,249.089307827,41.7307409658,-0.26635694576,-0.697129503289,0.665630853468,15320732877909,65.9565056197,42.2766225202,42.1645551696,31.3990957442,459561576148,-1.94055,-1,0.993865,0.00306749,0.00306749,0.0,0.00642404,0.00571097,14.2094,0.00364238,14.1972,0.0039325,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,14.1687,0.00331449,14.1747,0.00337262,14.1766,0.00426959,1.70204,0.0480872,135.19,0,--,-1,-1.37217,4194304,5,0.0,0.0
5,459598275341,46557,459561520602,249.088783225,41.8386617913,-0.265915361617,-0.695954643429,0.667035347448,15320905232307,66.1023436874,42.2785249258,42.0771202884,31.4843866085,459561576148,2.12365,1,0.00306749,0.993865,0.00306749,0.0,0.00766715,0.0068161,19.4787,0.331165,19.4787,0.331165,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,19.5737,0.190161,19.4502,0.217731,19.399,0.389935,0.814482,0.0608817,46.1616,0,--,1,1.50165,4194304,6,0.0,0.0
6,459598275342,46557,459561520602,249.088964626,41.8271266834,-0.265961083771,-0.696080917527,0.6668853411,15320904958065,66.0867570644,42.2782370872,42.0865237228,31.4753831739,459561576148,-5.07346,-1,0.993865,0.00306749,0.00306749,0.0,0.00752875,0.00669306,16.779,0.020396,16.7723,0.0292268,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,16.6917,0.0163059,16.711,0.0196059,16.7732,0.035773,1.43856,0.0924337,106.448,0,--,-1,-3.58748,4194304,7,0.0,0.0
7,459598275343,46557,459561520602,249.088172488,41.6673778105,-0.266633339987,-0.697811429524,0.66480513749,15320734629478,65.8708613999,42.2764969406,42.2152252045,31.347734619,459561576148,-0.125998,-1,0.993865,0.00306749,0.00306749,0.0,0.0058114,0.00516633,18.0646,0.0711287,18.0819,0.0991237,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999

In [43]:
log_file = open('log.txt', 'a')
for field in fields:
    field_moc = MOC(filename="../../dmu2/dmu2_field_coverages/{}_MOC.fits".format(field[0]))
    field_mask= inMoc(uhs["RA"],uhs["DEC"],field_moc)
    if not np.all( np.logical_not(field_mask)):
        uhs[field_mask].write("data/UHS-DR1_{}.fits".format(field[0]))
    
        subprocess.call(["stilts", 
                 "pixfoot", 
                 "in=./data/UHS-DR1_{}.fits".format(field[0]), 
                 "out=./data/UHS-DR1_{}_MOC.fits".format(field[0]),
                 "ra=RA",
                 "dec=DEC",
                 "order=12"], 
                    stdout=log_file)
